In [5]:
import numpy as np
import sklearn.neighbors as sk
import pickle
import time
import sys
sys.path.insert(0,"./../../periodic_kdtree/")
from periodic_kdtree import PeriodicCKDTree

In [6]:
#path="./../../../DLA Mock Catalogue/"
path="./../Data/"
names=["DLAhost_snap49_r1_b1 (copy).txt","DLAhost_snap49_r1_b1alpha (copy).txt","DLAhost_snap49_r1_b1T10 (copy).txt"]
boxsize=150.0
numPart=1024.0  # Number of particles will be the cube of this number
forceRes=(boxsize/numPart)/30  # Force resolution to avoid diverging values is usually 1/30th mean-interparticle spacing
maxLength=boxsize/4.0
#The maximum length upto which bins should be split is 1/5th to 1/10th the size of the box for the following reasons:
#- The boxes are periodic and any separation more than L/2 will require accessing other periodic boxes beyond the current one.
#- Of the available perturbation modes in the box, the K_min(biggest wavelength)= 2*pi/boxlength. There are modes below this but they are constrained by the size of the simulation box. Hence, another maxlength cutoff on length greater than K_min ~ L/(5 to 10)
#Being very flexible with this, I take the factor 4 for this run.
minLength=10.0*forceRes
eps=1.0     # need eps (epsilon) to avoid 0/0 ratio computations

In [7]:
def createBins(dR=2.0,mode='log',numOfBins=30):
    bins=[]
    if(mode!='log' and mode!='lin'):
        print("Issue with binning mode")
        exit()
        
    if(mode=='lin'):
        nBins=int(maxLength/dR)
        bins.append(np.arange(minLength,maxLength,dR))
        bins=bins[:-1]    # erasing the last entry because screwed up in cpp computation. Also unimportant because result=0.
    else:
        limit=np.log10(maxLength/minLength)
        marks=np.linspace(0.0,limit,numOfBins)
        bins=minLength*(np.power(10,marks))
    return bins

In [8]:
# Periodic correlations using github.com/Vidhate/periodic_kdtree
# Usage given inside in the readMe
def periodicCorrPP(pos,bins):
    bounds=np.array([boxsize,boxsize,boxsize])
    T=PeriodicCKDTree(bounds,pos)
    c=[]
    for b in bins[0:]:
        count=0
        for x in pos:
            neighbors=T.query_ball_point(x,b)
            count+=len(neighbors)
        c.append(count)
    c=np.array(c)
    return (c[1:]-c[:-1])

In [12]:
def periodicCorrelation(pos,dR=2.0):
    bins=createBins(mode="log",numOfBins=30)
    print("Bins created successfully")
    rpos=np.random.rand(len(pos),3)*boxsize
    print("Random points generated successfully")
    start=time.time()
    print("Starting pair counts using Periodic KDTRee")
    n_rr=np.array(periodicCorrPP(rpos,bins),dtype='double')+eps
    n_dd=np.array(periodicCorrPP(pos,bins),dtype='double')+eps
    centeredBins=bins[:-1]*0.5+bins[1:]*0.5
    correlationValue_pKDT=(n_dd/n_rr)-1.0
    
    end=time.time()
    print("Computed 2pt Correlation using PKDTree in "+str(end-start)+" seconds")

    return centeredBins,correlationValue_pKDT

In [10]:
# Reference : https://stackoverflow.com/questions/6159900/correct-way-to-write-line-to-file
def writeFile(bins,corrs,index):
    outname="PKDTree "+names[index]
    with open(outname,'w') as o:
        o.write("# PKDTree 2 point correlation for input file:"+names[index]+"\n")
        o.write("# Separation[R]     Correlation Value[Xi(R)]\n")
        for b,c in zip(bins,corrs):
            o.write(str(b)+" "+str(c)+"\n")
    print("Written to file")
    print("\n")

In [11]:
for i in range(0,len(names)):
    fname=path+names[i]+"_pickled"
    pos=pickle.load(open(fname,'rb'))
    b,c=periodicCorrelation(pos)
    writeFile(b,c,i)

Bins created successfully
Random points generated successfully
Starting pair counts using Periodic KDTRee
Computed 2pt Correlation using KDTree in 27.033302545547485 seconds
Written to file


Bins created successfully
Random points generated successfully
Starting pair counts using Periodic KDTRee
Computed 2pt Correlation using KDTree in 7.07572078704834 seconds
Written to file


Bins created successfully
Random points generated successfully
Starting pair counts using Periodic KDTRee
Computed 2pt Correlation using KDTree in 59.016371726989746 seconds
Written to file


